# Initialization

In [1]:
!spark-shell --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.4.1
      /_/
                        
Using Scala version 2.12.17, OpenJDK 64-Bit Server VM, 17.0.8
Branch HEAD
Compiled by user centos on 2023-06-19T23:01:01Z
Revision 6b1ff22dde1ead51cbf370be6e48a802daae58b6
Url https://github.com/apache/spark
Type --help for more information.


In [2]:
!pip install --quiet minio

# Titanic Logistic Regression using PySpark
This is a code along of the famous titanic dataset, its always nice to start off with this dataset because it is an example you will find across pretty much every data analysis language.

# Download data from public

In [3]:
!wget https://raw.githubusercontent.com/meddash-cloud/meddash-public-datasets/main/data/walmart_stock.csv

--2023-08-30 13:36:49--  https://raw.githubusercontent.com/meddash-cloud/meddash-public-datasets/main/data/hack_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14268 (14K) [text/plain]
Saving to: ‘hack_data.csv’

hack_data.csv       100%[===================>]  13.93K  --.-KB/s    in 0s      

2023-08-30 13:36:49 (73.6 MB/s) - ‘hack_data.csv’ saved [14268/14268]



# Upload the CSV file to Minio

In [1]:
BUCKET_NAME="spark-stream-csv-minio-01"
minio_url = "minio-service.kubeflow.svc.cluster.local:9000"
minio_key = "minio"
minio_secret = "minio123"

In [2]:
csv_file_name = "walmart_stock.csv"

In [3]:
from minio import Minio
from minio.error import S3Error
import os

upload_file_name=csv_file_name
upload_file_path="./{}".format(upload_file_name)
config = {
    "endpoint": minio_url,
    "access_key": "minio",
    "secret_key": "minio123",
    "secure": False,
    }


# Create a client with the MinIO server playground, its access key
# and secret key.
print ("connecting to minio {}".format(minio_url))
minio_client = Minio(**config)

print("try to find bucket {}".format(BUCKET_NAME))
found = minio_client.bucket_exists(BUCKET_NAME)
print("found", found)
if not found:
    minio_client.make_bucket(BUCKET_NAME)
else:
    print("Bucket '{}' already exists".format(BUCKET_NAME))

print("upload file to minio...")
minio_client.fput_object(BUCKET_NAME, os.path.basename(upload_file_path), upload_file_path)

connecting to minio minio-service.kubeflow.svc.cluster.local:9000
try to find bucket spark-stream-csv-minio-01
found False
upload file to minio...


FileNotFoundError: [Errno 2] No such file or directory: './hack_data.csv'

In [7]:
from pyspark.sql import SparkSession

In [7]:
minio_file_path="s3a://{}".format(BUCKET_NAME)

In [8]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import os

cwd = os.getcwd()

minio_url = "minio-service.kubeflow.svc.cluster.local:9000"
spark = SparkSession\
.builder\
.appName("ReadTextFilesFromS3")\
.master("local[*]")\
.config('spark.jars.packages','org.apache.hadoop:hadoop-aws:3.3.4')\
.config("spark.hadoop.fs.s3a.endpoint", "http://"+minio_url)\
.config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
.config("spark.hadoop.fs.s3a.path.style.access", "true")\
.config("spark.hadoop.fs.s3a.access.key", "minio")\
.config("spark.hadoop.fs.s3a.secret.key", "minio123")\
.getOrCreate()



sc = spark.sparkContext


